In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import removal, tokenization, stemming

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pipeline = Pipeline(collection = '../data/raw/collection.pkl', queries = '../data/raw/queries.pkl', features ='data/processed/qID_pID')

In [2]:
pipeline = Pipeline()

In [3]:
pipeline.setup()

In [13]:
pipeline.collection

,pID,Passage
npartitions=61,,
,int64,object
,...,...
...,...,...
,...,...
,...,...


In [4]:
pipeline.preprocess()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/dask/dataframe/core.py:3930: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Passage', 'object'))

  warnings.warn(meta_warning(meta))
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/dask/dataframe/core.py:3930: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  Af

AttributeError: 'WordListCorpusReader' object has no attribute '_LazyCorpusLoader__args'

In [ ]:
pipeline.tfidf()